<a href="https://colab.research.google.com/github/jaipandotco2525/python/blob/master/Cheminfo_rdkit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chem_info Project Based Learning 2019/09/21
# chem_project team2


### RDKIt

[RDKit](https://www.rdkit.org/) is a Open-Source Cheminformatics Software, which is very powerful and useful toolkit, however, it is not installed Google Colaboratory.

Thus, you need install rdkit packages first. Since Google Colab is a virtual server which is initialized for every session, you have to install every session.

Run the commands below. It will take a few minute. Please be patient.


In [0]:
!wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!time bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
!time conda install -q -y -c conda-forge rdkit

Now, setup the environments.

In [0]:
import sys
import os
sys.path.append('/usr/local/lib/python3.7/site-packages')

If the all packages have installed properly, the following text will be appeared in the next cell.

`see Chem/index.html in the doc tree for documentation`

In [0]:
import numpy as np
import pandas as pd
import rdkit
from rdkit import Chem
# from rdkit import Chem

print(Chem.__doc__)

Prepare some more tools.

In [0]:
from numpy import vectorize as vec
import scipy as sp
import sklearn
from sklearn.model_selection import train_test_split
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Descriptors,PandasTools


### Tox21 data set

Download a datafile `na-ar.sdf` from the link below.

https://drive.google.com/uc?id=1ALUNDgAmpauEa0Oa6-6TOPAG_uDTy0HH&export=download

Download a datafile `washed-nr-er.tsv` from the link below.

https://docs.google.com/uc?id=1N-Qykm94hz8NFPppYgTYcQwiRhqJNimw&export=download

Run the next cell. You will be ask to choose a file to upload, so upload the downloaded file from your local machine.

In [0]:
from google.colab import files
uploaded = files.upload()

Run the next cell to confirm the file is correctly uploaded. 

If it is uploaded, file size and file name will be shown.

In [0]:
! ls -la

Next, read the uploaded molecule data.

(Ignore the ERRORs on reading the file)

 `na-ar.sdf` is the original dataset.

`washed-nr-er.tsv` is a curated dataset. Unproper entries such as duplicate entries, inorganic molecules etc. has removed.

In [0]:
mols=PandasTools.LoadSDF('nr-ar.sdf',smilesName='SMILES',molColName='Molecule',includeFingerprints=True)


In [0]:
mols_washed = pd.read_csv('washed-nr-er.tsv')


Well, it's good to check detail data structures anytime you load new data.

In [0]:
mols.shape

mols_washed.shape

In [0]:
mols.columns

`head()` function is good method to browse a large data. 

In [0]:
mols[['Active', 'DSSTox_CID', 'FW', 'Formula', 'ID', 'Molecule', 'SMILES']].head(2)

Well, this data was originaly obtained from Toxicology Testing in the 21st Century ([Tox21](https://www.epa.gov/chemical-research/toxicology-testing-21st-century-tox21) ) database, federal collaboration among EPA, NIH, including National Center for Advancing Translational Sciences and the National Toxicology Program at the National Institute of Environmental Health Sciences, and the Food and Drug Administration.

They have been studied biological activity of thousands compounds to evaluate their toxicity and risk for human health.

In this database, you can see the molecular formula, molecular structure, and also, binary data whether the molecule has toxic activity or not.

In [0]:
mols['Active'][0:4]

In [0]:
print("Number of   active molecules:  ", list(mols.Active).count('1'))  #Active
print("Number of inactive molecules:  ", list(mols.Active).count('0'))  #Inactive

So, it would be nice if you can "predict" toxicity from molecular structure using some machine learning model. It will help to avoid dangerous chemicals and to find candidates of new drugs.

However, molecular structures are so complex so that you would convert them into some numeric representation, so called molecular descriptors.

Actually, there have been hundreds of "molecular descriptors" and it's still growing.

Fortunately, some of them are already implemented in RDKit library.

In [0]:
names = [x[0] for x in Descriptors._descList]
print("Number of descriptors in the rdkit: ", len(names))
np.array(names)[0:10]


In [0]:
for desc in ['TPSA','MaxPartialCharge','SlogP_VSA1','EState_VSA1','SMR_VSA1','MolLogP','MolMR','BalabanJ','Ipc','HallKierAlpha','Kappa1','Kappa2','Kappa3','RingCount','NumHAcceptors','NumHDonors']:
    exec("mols[desc]=vec(Descriptors.{})(mols['Molecule'])".format(desc))
print("shape of data : {}".format(mols.shape))


In [0]:
print(mols.isnull().any())

In [0]:
mols_desc=mols.drop(["MaxPartialCharge","Ipc","Kappa3", "Active", "DSSTox_CID", "FW", "Formula", "ID", "Molecule", "SMILES"], axis=1)

Let's see the distribution and correlation of some molecular features.

In [0]:
def set_color(L):
  tmp = []
  for l in L:
    if l == '1':
      tmp.append("red")
    else:
      tmp.append("palegreen")
  return tmp

pd.plotting.scatter_matrix(mols_desc,figsize=(16,16), hist_kwds={'bins':15},  
                           marker='+', s=8, alpha=.5, c=set_color(mols.Active))
plt.show()

### Classifier

As for an exmaple, apply Random Forest Classifiler to classify the molecules by toxic or not.

For validation, it is important to separate training data and test data.

In [0]:
X_train, X_test, y_train, y_test = train_test_split(mols_desc, mols.Active, train_size=0.75, test_size=0.25)

In [0]:
print("Training Data")
print("Number of   active molecules:  ", list(y_train).count('1'))
print("Number of inactive molecules:  ", list(y_train).count('0'))
print("Test Data")
print("Number of   active molecules:  ", list(y_test).count('1'))
print("Number of inactive molecules:  ", list(y_test).count('0'))

### Random Forest

Actrually, algorithms of [Random Forest](https://en.wikipedia.org/wiki/Random_forest) is not very simple. I would not explain in detail here.

Fortunately, this method is implemented in [scikit-learn](https://scikit-learn.org/stable/) library.

It's easy to use. Give the training and target data to the model.

In [0]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=0)
model.fit(X_train, y_train)


Now evaluate the results.

It isn't bad, is it?

In [0]:
print("Accuracy on training set: {:.3f}".format(model.score(X_train, y_train)))
print("Accuracy on test set:     {:.3f}".format(model.score(X_test , y_test)))
df = pd.DataFrame(model.feature_importances_, index = X_train.columns)
df

In scikit-learn library there are hundreds of tools and algorithms have been implemented. You may choose other models and methods to predict / classify molecules using chemical descriptors.

There is no magic algorithm that can solve every problems of machine learning. 
